In [1]:
import pandas as pd

In [4]:
Main_Data = pd.read_json("/Users/farhanrashid/Downloads/Amazon Data file(Books)/AMAZON_FASHION.json" , lines =True)


Main_Data.head(5)





,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,2,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,3,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN


In [5]:


# Define file paths
file_paths = {
    "5_star": "/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_5_star.csv",
    "4_star": "/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_4_star.csv",
    "2_star": "/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_2_star.csv",
    "1_star": "/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_1_star.csv",
    "common_4_and_5": "/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/common_reviews_4_and_5_stars.csv",
    "common_1_and_2": "/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/common_reviews_1_and_2_stars.csv"
}

# Function to create and save smaller and testing datasets
def create_datasets(file_path, smaller_file_path, testing_file_path):
    df = pd.read_csv(file_path)
    
    # Create smaller dataset (first 20 rows)
    smaller_df = df.head(20)
    
    # Create testing dataset (rest of the rows)
    testing_df = df.iloc[20:]
    
    # Save the smaller and testing datasets
    smaller_df.to_csv(smaller_file_path, index=False)
    testing_df.to_csv(testing_file_path, index=False)

# Generate the smaller and testing datasets for each file
for key, file_path in file_paths.items():
    smaller_file_path = file_path.replace(".csv", "_smaller.csv")
    testing_file_path = file_path.replace(".csv", "_testing.csv")
    
    create_datasets(file_path, smaller_file_path, testing_file_path)

print("Datasets created and saved successfully.")


Datasets created and saved successfully.


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Load your datasets
data_1_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_1_star_testing.csv')
data_2_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_2_star_testing.csv')
data_4_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_4_star_testing.csv')
data_5_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_5_star_testing.csv')

# Combine datasets
data = pd.concat([data_1_star, data_2_star, data_4_star, data_5_star])


data['reviewText'] = data['reviewText'].fillna('')

# Assuming 'review' is the text column and 'rating' is the label column
X = data['reviewText']
y = data['overall']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [17]:
print("Shape of X_train_tfidf:", X_train_tfidf.shape)
print("Shape of X_test_tfidf:", X_test_tfidf.shape)


Shape of X_train_tfidf: (629220, 10000)
Shape of X_test_tfidf: (157305, 10000)


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Train the model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

              precision    recall  f1-score   support

           1       0.68      0.78      0.73     21321
           2       0.46      0.26      0.34     12985
           4       0.58      0.36      0.44     29774
           5       0.81      0.93      0.87     93225

    accuracy                           0.75    157305
   macro avg       0.63      0.58      0.59    157305
weighted avg       0.72      0.75      0.72    157305



In [20]:
import joblib
joblib.dump(model, 'logistic_regression_model_2.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer_2.pkl')

['tfidf_vectorizer_2.pkl']

In [21]:
import joblib

# Load the trained model and vectorizer
model = joblib.load('logistic_regression_model_2.pkl')
vectorizer = joblib.load('tfidf_vectorizer_2.pkl')

# Function to preprocess and predict the star rating for a new review
def predict_star_rating(review_text, summary_text, vectorizer, model):
    # Combine reviewText and summary
    combined_text = review_text + ' ' + summary_text
    
    # Transform the combined text using the trained TF-IDF vectorizer
    text_tfidf = vectorizer.transform([combined_text])
    
    # Predict the star rating using the trained model
    predicted_rating = model.predict(text_tfidf)
    
    return predicted_rating[0]

# Example new review and summary
new_review_text = "Love these... I am going to order another pack to keep in work; someone (including myself) is always losing the back to an earring.  I don't understand why all fish hook earrings don't have them.  Just wish that they were a tiny bit longer.  :)"
new_summary_text = "My New 'Friends' !!"

# Preprocess and predict the star rating
predicted_rating = predict_star_rating(new_review_text, new_summary_text, vectorizer, model)
print(f"Predicted Star Rating: {predicted_rating}")


Predicted Star Rating: 5




# gradient boastng



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import joblib

# Load your datasets
data_1_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_1_star_testing.csv')
data_2_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_2_star_testing.csv')
data_4_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_4_star_testing.csv')
data_5_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_5_star_testing.csv')
data_common_4_and_5 = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/common_reviews_4_and_5_stars_testing.csv')
data_common_1_and_2 = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/common_reviews_1_and_2_stars_testing.csv')

# Combine datasets
data = pd.concat([data_1_star, data_2_star, data_4_star, data_5_star])

# Fill NaN values in 'reviewText' with empty strings
data['reviewText'] = data['reviewText'].fillna('')

# Create user-level features (example: user review count)
user_review_counts = data['userID'].value_counts().to_dict()
data['user_review_count'] = data['userID'].map(user_review_counts)

# Define feature set and labels
X = data[['reviewText', 'user_review_count']]
y = data['overall']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization for the text feature
vectorizer = TfidfVectorizer(max_features=10000)
X_train_text_tfidf = vectorizer.fit_transform(X_train['reviewText'])
X_test_text_tfidf = vectorizer.transform(X_test['reviewText'])

# Combine TF-IDF text features with user-level features
import scipy
X_train_combined = scipy.sparse.hstack((X_train_text_tfidf, X_train[['user_review_count']].values))
X_test_combined = scipy.sparse.hstack((X_test_text_tfidf, X_test[['user_review_count']].values))

# Train Gradient Boosting model
model = GradientBoostingClassifier()
model.fit(X_train_combined, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_combined)
print(classification_report(y_test, y_pred))

# Save the trained model and vectorizer
joblib.dump(model, 'gradient_boosting_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


In [ ]:
import joblib
import scipy

# Load the trained model and vectorizer
model = joblib.load('gradient_boosting_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Function to preprocess and predict the star rating for a new review
def predict_star_rating(review_text, user_review_count, vectorizer, model):
    # Transform the review text using the trained TF-IDF vectorizer
    text_tfidf = vectorizer.transform([review_text])
    
    # Combine TF-IDF text features with user-level features
    user_features = scipy.sparse.csr_matrix([[user_review_count]])
    combined_features = scipy.sparse.hstack((text_tfidf, user_features))
    
    # Predict the star rating using the trained model
    predicted_rating = model.predict(combined_features)
    
    return predicted_rating[0]

# Example new review
new_review_text = "This product is really amazing and exceeded my expectations."
new_user_review_count = 10  # Example user feature

# Preprocess and predict the star rating
predicted_rating = predict_star_rating(new_review_text, new_user_review_count, vectorizer, model)
print(f"Predicted Star Rating: {predicted_rating}")


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import joblib
import scipy

# Load your datasets
data_1_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_1_star_testing.csv')
data_2_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_2_star_testing.csv')
data_4_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_4_star_testing.csv')
data_5_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_5_star_testing.csv')
data_common_4_and_5 = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/common_reviews_4_and_5_stars.csv')
data_common_1_and_2 = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/common_reviews_1_and_2_stars.csv')

# Combine datasets
data = pd.concat([data_1_star, data_2_star, data_4_star, data_5_star])

# Fill NaN values in 'reviewText' with empty strings
data['reviewText'] = data['reviewText'].fillna('')

# Define feature set and labels
X = data['reviewText']
y = data['overall']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization for the text feature
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train Gradient Boosting model
model = GradientBoostingClassifier()
model.fit(X_train_tfidf, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

# Save the trained model and vectorizer
joblib.dump(model, 'gradient_boosting_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype

              precision    recall  f1-score   support

           1       0.66      0.57      0.61     21321
           2       0.44      0.08      0.13     12985
           4       0.54      0.29      0.37     29774
           5       0.73      0.95      0.82     93225

    accuracy                           0.70    157305
   macro avg       0.59      0.47      0.49    157305
weighted avg       0.66      0.70      0.65    157305



['tfidf_vectorizer.pkl']

In [23]:
import joblib
import scipy

# Load the trained model and vectorizer
model = joblib.load('gradient_boosting_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Function to preprocess and predict the star rating for a new review
def predict_star_rating(review_text, vectorizer, model):
    # Transform the review text using the trained TF-IDF vectorizer
    text_tfidf = vectorizer.transform([review_text])
    
    # Predict the star rating using the trained model
    predicted_rating = model.predict(text_tfidf)
    
    return predicted_rating[0]

# Example new review
new_review_text = "Love these... I am going to order another pack to keep in work; someone (including myself) is always losing the back to an earring.  I don't understand why all fish hook earrings don't have them.  Just wish that they were a tiny bit longer.  :)"

# Preprocess and predict the star rating
predicted_rating = predict_star_rating(new_review_text, vectorizer, model)
print(f"Predicted Star Rating: {predicted_rating}")


Predicted Star Rating: 5


In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import joblib
import scipy

# Load your datasets
data_1_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_1_star_testing.csv')
data_2_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_2_star_testing.csv')
data_4_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_4_star_testing.csv')
data_5_star = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/dataset_5_star_testing.csv')
data_common_4_and_5 = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/common_reviews_4_and_5_stars.csv')
data_common_1_and_2 = pd.read_csv('/Users/farhanrashid/Downloads/Amazon Data (BOOKS)/common_reviews_1_and_2_stars.csv')

# Combine datasets
data = pd.concat([data_1_star, data_2_star, data_4_star, data_5_star])

# Fill NaN values in 'reviewText' with empty strings
data['reviewText'] = data['reviewText'].fillna('')

# Create user-level features (example: user review count)
user_review_counts = data['reviewerID'].value_counts().to_dict()
data['user_review_count'] = data['reviewerID'].map(user_review_counts)

# Define feature set and labels
X = data[['reviewText', 'user_review_count']]
y = data['overall']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization for the text feature
vectorizer = TfidfVectorizer(max_features=10000)
X_train_text_tfidf = vectorizer.fit_transform(X_train['reviewText'])
X_test_text_tfidf = vectorizer.transform(X_test['reviewText'])

# Combine TF-IDF text features with user-level features
X_train_combined = scipy.sparse.hstack((X_train_text_tfidf, X_train[['user_review_count']].values.reshape(-1, 1)))
X_test_combined = scipy.sparse.hstack((X_test_text_tfidf, X_test[['user_review_count']].values.reshape(-1, 1)))

# Train Gradient Boosting model
model = GradientBoostingClassifier()
model.fit(X_train_combined, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_combined)
print(classification_report(y_test, y_pred))

# Save the trained model and vectorizer
joblib.dump(model, 'gradient_boosting_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/farhanrashid/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype

              precision    recall  f1-score   support

           1       0.66      0.57      0.61     21321
           2       0.44      0.08      0.13     12985
           4       0.54      0.29      0.37     29774
           5       0.73      0.95      0.82     93225

    accuracy                           0.70    157305
   macro avg       0.59      0.47      0.49    157305
weighted avg       0.66      0.70      0.65    157305



['tfidf_vectorizer.pkl']